Tesseract learning

In [7]:
from PIL import Image
import pytesseract

print(pytesseract.image_to_string(Image.open('image.png')))

NO. Item No Description Hscode P/O# S/O# Q'ty(Pcs) Unit Price(USD) Amount(USD)
2 104-10001 NeoNatalie Basic (dark) 902300 | TN200259 | S0125175 20 124.00 2480.00
10 340-00333 MamaNatalie Compl Light 902300 | TN200259 | S0125175 10 450.00 4500.00
1 340-00533 MamaN Compl (LT) Mama-U 902300 | TN200259 | SO0125175 1 705.60 705.60
1 340-11033 NeoNatalie Spare Dk 902300 | TN200259 | SO0125175 15 51.00 765.00
1 340-11133 NeoNatalie Spare Lt 902300 | TN200259 | S0125175 3 51.00 153.00
1 340-12033 NeoNatalie body (Qt2) Dk 902300 | TN200259 | SO0125175 36.60 36.60
38 360-00133 MamaBirthie (light) 902300 | TN200259 | SO0125175 38 586.00 22268.00
8 450-00033 LM Mama-U 902300 | TN200259 | SO0125175 36 153.00 5508.00




In [8]:
# print(pytesseract.image_to_data(Image.open('image.png')))

In [12]:
# print(pytesseract.image_to_boxes(Image.open('image.png')))

the coordinates of the characters are shown there

okay, pytesseract did the job right but it works better when the image is preprocessed,ie clean, high contrast image, so let's use openCV for image preprocessing

In [129]:
import cv2
import pytesseract
from PIL import Image

image_path = "image.png"
image = cv2.imread(image_path)
# print(image)

# convert to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# threshold is applied to make the important portion of the image to be prioritized
# _, thresh_image = cv2.threshold(gray_image, 150, 255, cv2.THRESH_BINARY)

thresh_image = cv2.adaptiveThreshold(gray_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                     cv2.THRESH_BINARY, 11, 2)
                                     

# optional step, just to check how image becomes
cv2.imwrite("text_detection.png", thresh_image)

# pytesseract only works with RGB image so let's convert
pil_image = Image.fromarray(cv2.cvtColor(thresh_image, cv2.COLOR_GRAY2RGB))

text = pytesseract.image_to_string(pil_image)

print(text)


NO Item No Description Hscode P/O# S/O# Q'ty(Pcs) Unit Price(USD) Amount(USD)
2

104-10001 NeoNatalie Basic (dark) 902300 TN200259 | SO125175 20 124.00 2480.00
io 340-00333 MamaNatalie Compl Light 902300 TN200259 | SO0125175 10 450.00 4500.00
1 340-00533 MamaN Compl (LT) Mama-U 902300 TN200259 | SO125175 a 705.60 705.60
1 340-11033 NeoNatalie Spare Dk 902300 TN200259 | SO0125175 a5 51.00 765.00
1 340-11133 NeoNatalie Spare Lt 902300 TN200259 | SO125175 3 51.00 153.00
1 340-12033 NeoNatalie body (Qt2) Dk 902300 TN200259 | SO0125175 1 36.60 36.60
38 360-00133 MamaBirthie (light) 902300 TN200259 | SO125175 38 586.00 22268.00
8 450-00033 LM Mama-U 902300 TN200259 | SO0125175 36 153.00 5508.00




let's save this output in a file for later use

In [71]:
with open("raw_text.txt", "w") as file:
    file.write(text)

now, we need to detect the table in the raw text, so that we can disect data as headers and ......

let's preprocess the image, now to detect the table, lines and ....

In [130]:

import numpy as np

image_path = "image.png"
image = cv2.imread(image_path)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY_INV, 15, 2)

horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 1))  # (Width, Height)
thresh_image = cv2.morphologyEx(thresh_image, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

# just for checking image 
cv2.imwrite("lines_detection.png", thresh)

True

detecting lines in table

In [245]:
edges = cv2.Canny(thresh, 50, 150, apertureSize=3)

# Detect lines using Hough Transform
# lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=20, 
#                         minLineLength=265, maxLineGap=10)


lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=155, 
                        minLineLength=261, maxLineGap=14)

# print(lines)
image_with_lines = image.copy()

for line in lines:
    x1, y1, x2, y2 = line[0]
    cv2.line(image_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)
    

cv2.imwrite("lines_detected.png",image_with_lines)

True

In [ ]:
# # print(lines)
# image_with_lines = image.copy()

# for line in lines:
#     x1, y1, x2, y2 = line[0]
#     cv2.line(image_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)
    

In [ ]:
# cv2.imwrite("lines_detected.png",image_with_lines)

True

now that lines are detected, we need to separate vertical and horizotal lines

In [158]:
print(lines[0])
# a bit confused which line's cordinate are these

[[   1  164 1979  164]]


In [246]:
# we have values in lines

# horizontal_lines = []
# vertical_lines = []

# for line in lines:
#     x1, y1, x2, y2 = line[0]

#     # Horizontal lines have a small y-difference (close to 0 slope)
#     if abs(y2 - y1) < 10:  # Threshold for horizontal
#         horizontal_lines.append([x1, y1, x2, y2])

#     # Vertical lines have a small x-difference (close to infinite slope)
#     elif abs(x2 - x1) < 10:  # Threshold for vertical
#         vertical_lines.append([x1, y1, x2, y2])

import numpy as np

horizontal_lines = []
vertical_lines = []

line_threshold = 10  # Distance threshold for merging lines
min_line_length = 50  # Filter out very short lines

for line in lines:
    x1, y1, x2, y2 = line[0]

    # Horizontal lines detection
    if abs(y2 - y1) < 10:  # Close to 0 slope
        if abs(x2 - x1) > min_line_length:  # Ignore small lines
            horizontal_lines.append([x1, y1, x2, y2])

    # Vertical lines detection
    elif abs(x2 - x1) < 10:  # Close to infinite slope
        if abs(y2 - y1) > min_line_length:  # Ignore small lines
            vertical_lines.append([x1, y1, x2, y2])

    # Sort lines
horizontal_lines = sorted(horizontal_lines, key=lambda x: x[1])  # Sort by y-coordinate
vertical_lines = sorted(vertical_lines, key=lambda x: x[0])  # Sort by x-coordinate

print(f"Detected {len(horizontal_lines)} horizontal lines and {len(vertical_lines)} vertical lines")

Detected 21 horizontal lines and 12 vertical lines


In [75]:
print(horizontal_lines[0])
# (2,0) (1980,0)

[np.int32(2), np.int32(0), np.int32(1980), np.int32(0)]


In [247]:

cells = []
for i in range(len(horizontal_lines) - 1):  # Iterate over rows
    for j in range(len(vertical_lines) - 1):  # Iterate over columns
        x1 = vertical_lines[j][0]  # Left x-coordinate
        y1 = horizontal_lines[i][1]  # Top y-coordinate
        x2 = vertical_lines[j + 1][0]  # Right x-coordinate
        y2 = horizontal_lines[i + 1][1]  # Bottom y-coordinate
        cells.append((x1, y1, x2, y2))

# Draw the cells on the image for visualization
image_with_cells = image.copy()
for (x1, y1, x2, y2) in cells:
    cv2.rectangle(image_with_cells, (x1, y1), (x2, y2), (0, 0, 255), 2)


cv2.imwrite("cells_detected.png", image_with_cells)

True

we detected the text in the very beginning, but then why are we doing these steps again??
well, that was a random text block, but we need the the text along with the cell information
,,, in which cell the data is so that we can form the json format according to that

In [249]:
cell_text = []
for idx, (x1, y1, x2, y2) in enumerate(cells):
    # Crop the cell from the original image
    cell_image = image[y1:y2, x1:x2]
    
    # Convert to RGB for PIL (pytesseract requires RGB)
    cell_image_rgb = cv2.cvtColor(cell_image, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(cell_image_rgb)
    
    # Extract text using pytesseract
    text = pytesseract.image_to_string(pil_image).strip()
    
    # Store the text and coordinates
    cell_text.append({"text": text, "coords": (x1, y1, x2, y2)})
    # print(f"Cell {idx + 1} Text: '{text}'")

# i will see later from here

In [252]:
# import json

# with open("cell_text.json", "w") as f:
#     json.dump(cell_text, f)

import json
import numpy as np

def default_converter(o):
    if isinstance(o, np.integer):
        return int(o)
    raise TypeError(f"Object {o} is not JSON serializable")

with open("cell_text.json", "w") as f:
    json.dump(cell_text, f, default=default_converter)

In [253]:
# Assume cell_text is already populated from the previous step
headers_raw = [cell["text"] for cell in cell_text[:8]]  # First row (8 cells)

# Process each header: lowercase, underscore-separated, remove non-alphabet
import re
headers = []
for header in headers_raw:
    # Convert to lowercase and replace non-alphabet with underscore
    processed = re.sub(r'[^a-z0-9]+', '_', header.lower()).strip('_')
    headers.append(processed if processed else "unknown")  # Avoid empty strings

print("Processed Headers:", headers)

Processed Headers: ['unknown', 'no', 'item_no', 'description', 'hscode', 'p_o', 's_o', 'unknown']


In [254]:
# Total cells and number of data rows
total_cells = len(cell_text)
data_cells = cell_text[8:]  # Exclude header row
rows = [data_cells[i:i + 8] for i in range(0, len(data_cells), 8)]

print(f"Total rows detected: {len(rows)}")
for idx, row in enumerate(rows):
    print(f"Row {idx + 1}: {[cell['text'] for cell in row]}")

Total rows detected: 27
Row 1: ["O'ty(Pcs)", 'Unit Price(USD)', 'Amount(USD)', '', '', '', '', '']
Row 2: ['', '', '', '', '', '', '', '']
Row 3: ['104-10001', 'NeoNatalie Basic (dark)', '902300', 'TN200259', '$0125175', '', '20', '124.00']
Row 4: ['2480.00', '', '10', '340-00333', 'MamaNatalie Comp! Light', '902300', 'TN200259', '$0125175']
Row 5: ['', '10', '450.00', '4500.00', '', '', '', '']
Row 6: ['', '', '', '', '', '', '', '']
Row 7: ['', '', '', '', '', '', '', '']
Row 8: ['', '', '', '', '', '', '', '']
Row 9: ['', '', '', '', '', '', '', '']
Row 10: ['', '', '', '', '', '', '', '']
Row 11: ['', '', '', '', '', '', '', '']
Row 12: ['', '', '', '', '', '', '', '']
Row 13: ['', '', '', '', '', '', '', '']
Row 14: ['', '', '', '', '', '', '', '']
Row 15: ['', '', '', '', '', '', '', '']
Row 16: ['', '', '', '', '', '', '', '']
Row 17: ['', '', '', '', '', '', '', '']
Row 18: ['', '', '', '', '', '', '', '']
Row 19: ['', '', '', '', '', '', '', '']
Row 20: ['', '', '', '', '', ''

In [255]:
# Create a list of dictionaries
json_data = []
for row in rows:
    row_dict = {}
    for header, cell in zip(headers, [cell["text"] for cell in row]):
        row_dict[header] = cell
    json_data.append(row_dict)

print("JSON Data:", json_data)

JSON Data: [{'unknown': '', 'no': 'Unit Price(USD)', 'item_no': 'Amount(USD)', 'description': '', 'hscode': '', 'p_o': '', 's_o': ''}, {'unknown': '', 'no': '', 'item_no': '', 'description': '', 'hscode': '', 'p_o': '', 's_o': ''}, {'unknown': '124.00', 'no': 'NeoNatalie Basic (dark)', 'item_no': '902300', 'description': 'TN200259', 'hscode': '$0125175', 'p_o': '', 's_o': '20'}, {'unknown': '$0125175', 'no': '', 'item_no': '10', 'description': '340-00333', 'hscode': 'MamaNatalie Comp! Light', 'p_o': '902300', 's_o': 'TN200259'}, {'unknown': '', 'no': '10', 'item_no': '450.00', 'description': '4500.00', 'hscode': '', 'p_o': '', 's_o': ''}, {'unknown': '', 'no': '', 'item_no': '', 'description': '', 'hscode': '', 'p_o': '', 's_o': ''}, {'unknown': '', 'no': '', 'item_no': '', 'description': '', 'hscode': '', 'p_o': '', 's_o': ''}, {'unknown': '', 'no': '', 'item_no': '', 'description': '', 'hscode': '', 'p_o': '', 's_o': ''}, {'unknown': '', 'no': '', 'item_no': '', 'description': '', 'h

In [256]:
# Save to JSON file
with open("output.json", "w") as f:
    json.dump(json_data, f, indent=4)

print("JSON file saved as 'output.json'")

JSON file saved as 'output.json'


In [258]:
import zipfile

# Create a zip file
with zipfile.ZipFile("submission.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    zipf.write("output.json")
    zipf.write("image.png")

print("Submission zip file created as 'submission.zip'")

Submission zip file created as 'submission.zip'
